In [1]:
import polars as pl
from methods import Config, RecombAnalysis

# Data required for analysis

Fill out the `config.yaml` file fields with the names of each of these files

- RIVET results file (.txt)
- RIVET vcf file (.vcf)
- Chronumental results file (.tsv)
- PyR0 mutations file (.tsv)
- Case counts file (.csv)
- Standing genetic diversity results file (.csv)



In [3]:
# Ensure correct path to config.yaml file in top level directory
CONFIG_FILEPATH = "../config.yaml"
# Get all the data files from the config file
config = Config(CONFIG_FILEPATH)

File not found: {} test_data/final_recombinants_2023-12-25.txt


SystemExit: 1

In [7]:
analysis = RecombAnalysis(config)

Loading all datasets for analysis
Loading standing genetic diversity scores from file:  ../test_data/genetic-diversity-gisaidAndPublic.2023-12-25.csv
Loading PyR0 amino acid mutation ranking scores from file:  ../test_data/mutations.tsv
Loading case count data from file:  ../test_data/time_series_covid19_confirmed_global.csv
Loading chronumental object from disk cache:  ../test_data/chronumental_dates_gisaidAndPublic.2023-12-25-STEPS2000-SERIAL3.metadata.tsv.tsv.pkl
Chronumental dates loaded.
Data loaded, analysis ready. Elapsed time: 10.4371 seconds


In [8]:
df = analysis.toDataframe()
df

Month,Count,Infections,NumRecombsDetectedByMonth
str,f64,i64,i64
"""2023-01""",119.41,10270797,28
"""2021-05""",140.826,19347136,83
"""2020-06""",70.4285,4292072,11
"""2021-10""",133.259,13064658,94
"""2021-09""",128.393,16061455,117
…,…,…,…
"""2021-11""",141.142,15702891,107
"""2022-09""",102.288,14721332,75
"""2022-03""",89.132,51347034,64


# Figure 1c: Correlation Matrix

In [9]:
selected_columns = ['Count', 'Infections', 'NumRecombsDetectedByMonth']
df.to_pandas()[selected_columns].corr()

,Count,Infections,NumRecombsDetectedByMonth
Count,1.000000,0.124601,0.779849
Infections,0.124601,1.000000,0.306916
NumRecombsDetectedByMonth,0.779849,0.306916,1.000000


In [10]:
print(df.select(pl.corr("Count", "NumRecombsDetectedByMonth")).item())
print(df.select(pl.corr("Infections", "NumRecombsDetectedByMonth")).item())
print(df.select(pl.corr("Infections", "Count")).item())

0.7798487013090407
0.30691635828778735
0.12460145765167176
